In [ ]:
#Testbed 2.3 LeakyReLU

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import time
import csv

In [13]:
#switching from ReLU to leakyReLU
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=512, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=10)
        self.dropout = nn.Dropout(p=0.5)
    def forward(self, x):
        x = nn.functional.leaky_relu(self.conv1(x), negative_slope=0.1)
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = nn.functional.leaky_relu(self.conv2(x), negative_slope=0.1)
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = nn.functional.leaky_relu(self.conv3(x), negative_slope=0.1)
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = nn.functional.leaky_relu(self.conv4(x), negative_slope=0.1)
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = nn.functional.leaky_relu(self.conv5(x), negative_slope=0.1)
        x = self.dropout(x)
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = nn.functional.leaky_relu(self.fc1(x))
        x = self.dropout(x)
        x = nn.functional.leaky_relu(self.fc2(x))
        #x = nn.functional.softmax(self.fc3(x))
        x = self.fc3(x)
        return x

In [14]:
# Define the dataset and data loader
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(10),
                                transforms.RandomCrop(32, padding=4),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [15]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
# Define the loss function and optimizer
model = SimpleModel()
#loss function is used to measure the error between the predicted output of the model and the ground-truth label
criterion = nn.CrossEntropyLoss() #LF
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# Define the number of training epochs
num_epochs = 100
patience = 99 #set to 99 for testing as we want to see long term model performance 
counter = 0

In [19]:
#TEMP load in model
#model = SimpleModel()
#model.load_state_dict(torch.load('CI1_model.pt'))
#results showed loading in ReLU model did not produce great loss results - weights are not really transferrable between different loss function models?

FileNotFoundError: [Errno 2] No such file or directory: 'CI1_model.pt'

In [19]:
# Train the model
valid_test = 99
epoch_test = 0
train_losses = []
test_losses = []
train_acc = []
test_acc = []
time_list = []
t1 = time.perf_counter()
print('Begin model training')
print('Patience has been set at {} for {} epochs'.format(patience, num_epochs))
for epoch in range(num_epochs):
    train_loss = 0
    test_loss = 0
    
    #Training 
    correct_t = 0
    total_t = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs) #tensor produced, indecipherable when printed 
        loss = criterion(outputs, labels) #criterion is loss function
        loss.backward() #used to calculate the gradients of the parameters of a model with respect to a loss function
        optimizer.step() #updates the model parameters based on the gradients computed during the backward pass of training
                
        _, preds = torch.max(outputs, 1) #produces tensor containing indices of the maximum values (i.e. the predicted classes)
        correct_t += (preds == labels).sum().item()
        train_loss += loss.item()
        total_t += labels.size(0) #total equals 50000 by the end of this for loop for CIFAR10
        #correct_t += (preds == labels).sum().item()
        #print(preds==labels)
        #print()
    
    #Validation
    correct_v = 0
    total_v = 0
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        correct_v += (preds == labels).sum().item()
        test_loss += loss.item()
        total_v += labels.size(0) #total equals 10000 by the end of this for loop for CIFAR10
        #add correct
   
    
    #defining accuracy and loss 
    t_acc = correct_t/total_t
    v_acc = correct_v/total_v
    new_train_lost = train_loss / len(train_loader)
    new_valid_lost = test_loss / len(test_loader)
    
    #Save trained model if it is improved 
    if new_valid_lost < valid_test:
        valid_test = new_valid_lost
        epoch_test = epoch+1
        torch.save(model.state_dict(), "leaky_relu_model2.3.1.pt")
        counter = 0
        print ('Temp model saved at Epoch {} with validation lost of {:.4f}'.format(epoch_test, valid_test))
    else:
        counter+=1
        print('No improvement in test, count is {}'.format(counter))
        
    #Terminates training after model stops improving based on patience 
    if counter > patience:
        break
    
    #prints stuff
    t2 = time.perf_counter()
    print('Training loss for Epoch {} is {:.4f} and Training accuracy is {:.2f}'.format(epoch + 1, new_train_lost, t_acc))
    print('Validation loss for Epoch {} is {:.4f} and Validation accuracy is {:.2f}'.format(epoch + 1, new_valid_lost, v_acc))
    print('Completed Epoch {} in {:.1f} seconds'.format(epoch + 1, t2-t1))
    #print('Epoch: {} Loss: {:.4f} Train_Acc: {:.4f}'.format(epoch, train_loss / len(dataloader), running_corrects.double() / len(dataset)))
    
    #makes list of loss, accuract, and time for epoch
    train_acc.append(t_acc)
    test_acc.append(v_acc)
    train_losses.append(new_train_lost)
    test_losses.append(new_valid_lost)
    time_list.append(t2-t1)

print ('Final model saved at Epoch {} with validation lost of {:.4f}'.format(epoch_test, valid_test))


Begin model training
Patience has been set at 99
Temp model saved at Epoch 1 with validation lost of 1.3888
Training loss for Epoch 1 is 1.6910 and Training accuracy is 0.38
Validation loss for Epoch 1 is 1.3888 and Validation accuracy is 0.50
Completed Epoch 1 in 134.6 seconds
Final model saved at Epoch 1 with validation lost of 1.3888


In [ ]:
# Write train and test lists to a CSV file
with open('leaky_relu_model2.3.1.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Training Loss', 'Test Loss', 'Train Accuracy', 'Test Accuracy', 'Time'])
    rows = zip(train_losses, test_losses, train_acc, test_acc, time_list)
    writer.writerows(rows)